In [27]:
# Part 3

In [1]:
# reading a file

def readfile(file):
    with open(file, encoding = 'utf-8') as f:
        
        # sort file into comments, with each comment being an element of the list
        l = f.read().strip().split('\n\n')
        
        # each word-line is then made into an element within each comment element
        l2 = [i.split('\n') for i in l]
        
        # each word in a word line is made the zeroth element of the 2nd nested list, and its respective entity/sentiment label the first element
        for idx, line in enumerate(l2):
            line = [i.split(" ") for i in line]
            l2[idx] = line
        return l2

data = readfile('train')
data[:2]
# print(data[0])

[[['Welcome', 'O'],
  ['lunch', 'O'],
  ['for', 'O'],
  ['Charlotte', 'B-neutral'],
  ['and', 'O'],
  ['Tin', 'B-neutral'],
  ['2', 'I-neutral'],
  ['.', 'O'],
  ['#DellPinoys', 'O'],
  ['#iworkfordell', 'O'],
  ['@', 'O'],
  ['THE', 'B-neutral'],
  ['CHEVRONS', 'I-neutral'],
  ['https://t.co/B5SpZKcsWq', 'O']],
 [["I'm", 'O'],
  ['at', 'O'],
  ['Virgo', 'O'],
  ['Function', 'O'],
  ['Rooms', 'O'],
  ['in', 'O'],
  ['Singapore', 'B-neutral'],
  ['https://t.co/fBbt0AQlTs', 'O']]]

In [3]:
import numpy as np

In [7]:
np.array(data[:10]).shape

(10,)

In [9]:
testing = data[0]
testing.insert(0,['','START'])
testing.append(['','STOP'])
print(testing)

[['', 'START'], ['Welcome', 'O'], ['lunch', 'O'], ['for', 'O'], ['Charlotte', 'B-neutral'], ['and', 'O'], ['Tin', 'B-neutral'], ['2', 'I-neutral'], ['.', 'O'], ['#DellPinoys', 'O'], ['#iworkfordell', 'O'], ['@', 'O'], ['THE', 'B-neutral'], ['CHEVRONS', 'I-neutral'], ['https://t.co/B5SpZKcsWq', 'O'], ['', 'STOP']]


In [6]:
print(len(testing))

16


In [36]:
for i, example in enumerate(data):
    if example == [['']]:
        print(i)

In [33]:
len(data)

18560

In [2]:
# count state transitions

def cstDict(data):
    
    # dictionary holding the count of unique transitions from state y_i-1 to y_i, i.e. from a word of a comment to the next
    # e.g. the if the transition 'machine' --> 'learning' is not in the dictionary already, tuple ('machine','learning') is added as a key with value 1
    dictTran = {}
    
    # dictionary holding the count of unique states y_i-1, with STOP not included
    dictY = {}
    
    for comments in data:
        
        # insert a start and stop state to each comment
        comments.insert(0,['', 'START'])
        comments.append(['','STOP'])

        counter = len(comments)
        
        idx = counter - 1
        idx2 = counter - 2
        
        # adding each state pair (y_i-1, y_i) to dictTran
        # print(comments)
        try:
            while idx >= 0:
                word_lower = comments[idx][1] # make words lower case
                # print(word_lower)
                # print(comments[idx - 1])
                prev_word_lower = comments[idx - 1][1]
                transition = (prev_word_lower, word_lower) # add to tuple
                if transition not in dictTran.keys(): # check if already in dict
                    dictTran[transition] = 1
                else:
                    dictTran[transition] += 1
                idx -= 1
                
            # adding each state y_i-1 to dictY
            while idx2 >= 0:
                prev_word_lower = comments[idx2][1] # make words lower case
                if prev_word_lower not in dictY.keys(): # check if already in dict
                    dictY[prev_word_lower] = 1
                else:
                    dictY[prev_word_lower] += 1
                idx2 -= 1
        except:
            print(comments)
            
    return dictTran, dictY

def cst(yi_1, yi):
    
    # get data from cstDict
    compare_tran, compare_yi_1 = cstDict(data)
    
    # change parameters to lower case
    yi = yi.lower()
    yi_1 = yi_1.lower()
    
    # add parameters to tuple to compare in dictionary
    tranKey = (yi_1, yi)
    
    # numerator and denominator
    num = compare_tran[tranKey]
    denom = compare_yi_1[yi_1]
    
    # compute probability
    prob = num/denom * 100
    prob_round = round(prob, 5)
    output = "The probability of '" + str(yi) + "' appearing after '" + str(yi_1) + "' is: " + str(prob_round) + "%"
    return output

In [3]:
# data = [
#     [['O', 'I'], ['like'], ['machine'], ['learning']],
#     [['I'], ['love'], ['machine'], ['learning']],
# ]
a = cstDict(data)
a

({('O', 'STOP'): 17988,
  ('I-neutral', 'O'): 12025,
  ('B-neutral', 'I-neutral'): 12267,
  ('O', 'B-neutral'): 23247,
  ('O', 'O'): 132947,
  ('B-neutral', 'O'): 11979,
  ('START', 'O'): 16697,
  ('STOP', 'START'): 18559,
  ('B-positive', 'O'): 2132,
  ('O', 'B-positive'): 3040,
  ('I-positive', 'O'): 1330,
  ('B-positive', 'I-positive'): 1391,
  ('START', 'B-positive'): 552,
  ('B-neutral', 'STOP'): 265,
  ('I-neutral', 'I-neutral'): 16493,
  ('START', 'B-neutral'): 1141,
  ('I-negative', 'O'): 300,
  ('B-negative', 'I-negative'): 315,
  ('O', 'B-negative'): 947,
  ('I-positive', 'I-positive'): 1167,
  ('B-negative', 'O'): 759,
  ('START', 'B-negative'): 169,
  ('B-neutral', 'B-neutral'): 340,
  ('I-positive', 'B-neutral'): 13,
  ('I-neutral', 'B-neutral'): 111,
  ('B-positive', 'STOP'): 75,
  ('B-negative', 'STOP'): 48,
  ('I-positive', 'STOP'): 43,
  ('B-neutral', 'B-negative'): 5,
  ('I-neutral', 'STOP'): 127,
  ('B-neutral', 'B-positive'): 12,
  ('I-negative', 'I-negative'): 186,

In [13]:
b = cst('I','like')
b

"The probability of 'like' appearing after 'i' is: 50.0%"

In [15]:
test1,test2 = a
print(test2['like'])

1
